## **Dataset Download**

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c tensorflow-speech-recognition-challenge
!7z x train.7z

In [ ]:
labels = [
    'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine'
]
train_audio_path = './train/audio/'

all_wave = []
all_label = []
for label in tqdm(labels):
    waves = [f for f in os.listdir(train_audio_path + label) if f.endswith('.wav')]
    i = 0
    for wav in waves:
        i += 1
        samples, sample_rate = librosa.load(train_audio_path + '/' + label + '/' + wav, sr = 16000)
        samples = librosa.resample(samples, sample_rate, 8192)
        if(len(samples)== 8192) :
            all_wave.append(samples)
            all_label.append(label)
        if i== 10:
            break

pro_wave = np.array(all_wave).reshape(-1,8192,1)
pro_label = np.array(all_label)
np.save('waves.npy', pro_wave)
np.save('labels.npy', pro_label)
np.random.shuffle(pro_wave)

In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip

from PIL import Image
image = Image.open('/content/val2017/000000000139.jpg')
print(image.format)
print(image.size)
print(image.mode)
plt.imshow(image)

train_img_path = './val2017/'
imgs = [f for f in os.listdir(train_img_path) if f.endswith('.jpg')]
# all_img = np.empty((len(imgs), 128, 128, 3))
all_img = []
for i in tqdm(range(len(imgs))):
  temp = Image.open(train_img_path + imgs[i])
  if(temp.mode == 'RGB'):
    temp = temp.resize((128, 128), Image.ANTIALIAS)
    temp = np.array(temp)
    temp = temp.astype(np.float32) / 255.0 
    all_img.append(temp)

In [ ]:
import random
random.seed(1)
all_img = all_img + random.sample(all_img, 10)
random.shuffle(all_img)
images = np.array(all_img)
np.save('img_dataset.npy', images)

## *Test Dataset*

In [2]:
class DataGenerator(object):
  def __init__(self, batch_size=32, seed=999):
    np.random.seed(seed)
    self.batch_size = batch_size
    self.audio_data = np.load('waves.npy')
    np.random.shuffle(self.audio_data)
    self.img_data = np.load('img_dataset.npy')
    self.img_data = self.img_data.astype(np.float32) / 255.0
    np.random.shuffle(self.img_data)
  
  def sample_batch(self):
    img_batch = np.random.choice(self.img_data.shape[0], self.batch_size, replace=False)
    img_batch = self.img_data[img_batch]
    audio_batch = np.random.choice(self.audio_data.shape[0], self.batch_size, replace=False)
    audio_batch = self.audio_data[audio_batch]
    return img_batch, audio_batch

In [ ]:
generator = DataGenerator()